In [4]:
#Chatbot Development
#Import nltk library
import nltk

In [ ]:
# GUI to download certain libraries
nltk.download_gui()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
#Import libraries
import re
import csv
import os
from nltk.stem.snowball import SnowballStemmer
from nltk.classify import SklearnClassifier
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import pandas as pd
import random

In [ ]:
#Import interactive shell to provide multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
#Ignore all warnings
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [ ]:
# Convert a sentence into lower case
# Tokenize the sentence and extract individual words
# Remove words that do not add meaning to the sentence
sentence1 = 'The Big brown box jumped over a fox'
def preprocess(sentence1):
    sentence1= sentence1.lower()
    tokenizer = RegexpTokenizer("\w+")
    tokens = tokenizer.tokenize(sentence1)
    filtered_words = [w for w in tokens if not w in stopwords.words('english')]
    return filtered_words

In [ ]:
#print preprocessed words
preprocess_sentence = preprocess(sentence1)

In [ ]:
tags = nltk.pos_tag(preprocess_sentence)
print(tags)

In [ ]:
def extra_tagged(sentences):
    feature = []
    for tagged_word in sentences:
        word, tag = tagged_word
        feature.append(word)
    return feature

In [ ]:
extra_tagged(tags)

In [ ]:
# lemmatize the word
lmtzr = WordNetLemmatizer()
print(lmtzr.lemmatize('dollars'))

In [ ]:
# Stemming the words
stemwords = ['stem', 'stems', 'stemming', 'foot', 'feeter']
stemmer  = SnowballStemmer('english')
[stemmer.stem(x) for x in stemwords]

In [ ]:
# Putting it all together
def extract_features(text):
    words = preprocess(text)
    tags  = nltk.pos_tag(words)
    extract_features = extra_tagged(tags)
    stemmed_words = [stemmer.stem(x) for x in extract_features]
    result = [lmtzr.lemmatize(x) for x in stemmed_words]
    return result

In [ ]:
sentence2 = 'he hurt is right foot while wearing a shoe on his lazy feet'
words = extract_features(sentence2)
print(words)

In [ ]:
# Bag of words
def word_feats(words):
    return dict([(word, True) for word in words])

In [ ]:
word_feats(words)

In [ ]:
# Parsing the document
def extract_feature_from_doc(data):
    result = []
    corpus = []
    # the responses of the chatbot
    answers = {}
    for text, category, answer in data:
        features = extract_features(text)
        corpus.append(features)
        result.append((word_feats(features), category))
        answers[category] = answer
    return (result, sum(corpus, []), answers)

In [ ]:
def get_content(filename):
    doc=os.path.join(filename)
    with open(doc,'r') as content_file:
        lines = csv.reader(content_file, delimiter='|')
        data = [x for x in lines if len(x) == 3]
        return data

In [ ]:
filename = 'C:/Lenin Data Science/leaves.txt'

In [ ]:
data = get_content(filename)

In [ ]:
data

In [ ]:
features_data, corpus, answers = extract_feature_from_doc(data)

In [ ]:
print(features_data[50])

In [ ]:
# Train model using this features
# split data into test and train
split_ratio = 0.8

In [ ]:
def split_dataset(data, split_ratio):
    random.shuffle(data)
    data_length = len(data)
    train_split = int(data_length * split_ratio)
    return(data[:train_split], data[train_split:])

In [ ]:
train_data, test_data = split_dataset(features_data, split_ratio)

In [ ]:
# Save the data
np.save('training_data', train_data)
np.save('test_data', test_data)

In [ ]:
training_data = np.load('training_data.npy', allow_pickle=True)
test_data = np.load('test_data.npy',allow_pickle=True)

In [ ]:
# Decision tree classifier
def train_decision_tree(training_data, test_data):
    classifier = nltk.classify.DecisionTreeClassifier.train(training_data, entropy_cutoff=0.6, support_cutoff=6)
    classifier_name = type(classifier).__name__
    training_set_accuracy = nltk.classify.accuracy(classifier, train_data)
    print('training set accuracy', training_set_accuracy)
    test_set_accuracy = nltk.classify.accuracy(classifier, test_data)
    print('test set accuracy', test_set_accuracy)
    return classifier, classifier_name, training_set_accuracy, test_set_accuracy

In [ ]:
dtclassifier, classifier_name, training_set_accuracy, test_set_accuracy = train_decision_tree(training_data, test_data)

In [ ]:
print(classifier_name)

In [ ]:
# Naive Bayes classifier
def Naive_Bayes_Classifier(training_data, test_data):
    classifier = nltk.NaiveBayesClassifier.train(training_data)
    classifier_name = type(classifier).__name__
    training_set_accuracy = nltk.classify.accuracy(classifier,training_data)
    print('training set accuracy', training_set_accuracy)
    test_set_accuracy = nltk.classify.accuracy(classifier,test_data)
    print('test set accuracy', test_set_accuracy)
    return classifier, classifier_name, training_set_accuracy, test_set_accuracy

In [ ]:
nbclassifier, classifier_name, training_set_accuracy, test_set_accuracy = Naive_Bayes_Classifier(training_data, test_data)

In [ ]:
print(len(nbclassifier.most_informative_features()))

In [ ]:
nbclassifier.show_most_informative_features()

In [ ]:
def reply(input_sentence):
    category = dtclassifier.classify(word_feats(extract_features(input_sentence)))
    return answers[category]


In [ ]:
reply("hi")

In [ ]:
reply("what leaves I have used")